In [1]:
import numpy as np
import pandas as pd

import requests
import pickle

from selenium import webdriver
from bs4 import BeautifulSoup

from time import sleep

from datetime import datetime

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

import re

In [3]:
def find_years_of_experience(string: str):
    
    list_strings = ["años de", "years of", "years experience", "años experiencia"]
    
    string = string.lower()
    
    years = [string[string.find(s) - 5 : string.find(s) + len(s) + 1] for s in list_strings if string.find(s) != -1]
    
    numeros = [re.findall(r"\d+", y) for y in years]

    numeros = [[int(n) for n in num if 0 < int(n) < 13] for num in numeros]
    
    numeros = [max(num) if num else np.nan for num in numeros]

    return max(numeros) if numeros else np.nan


def experience_level(num):
    
    if not pd.isna(num):
    
        if num < 2:
            return "Junior"
        elif num <= 4:
            return "Semi-Senior"
        elif num < 8:
            return "Senior"
        else:
            return "Leader"
        
    else:
        return np.nan
    
    
def get_especialidad(string, list_especialidad):
    
    especialidades = list({especialidad for especialidad in list_especialidad if especialidad.lower() in string.lower()})
    
    return especialidades if especialidades else []

def get_perfil(lista, dict_perfiles):
    
    perfiles = list({v for k, v in dict_perfiles.items() if k in lista})
    
    return perfiles if perfiles else []

In [4]:
df_perfil_especialidad = pd.read_csv("StackMinimo - StackTec.csv", skiprows = 2).drop("Unnamed: 0", axis = 1)

list_especialidad = ["Backend",
                     "Base de datos",
                     "Bases de datos",
                     "APIs",
                     "Mobile",
                     "Frontend Movil",
                     "Frontend Web",
                     "Frontend"
                     "Full Stack",
                     "Integración",
                     "Liferay",
                     "Power BI",
                     "Software Release Engineer",
                     "Diseñador Gráfico",
                     "Infraestructura cloud",
                     "Sistemas Operativos",
                     "Seguridad",
                     "Mantenimiento y Soporte",
                     "Administrador de Base de datos",
                     "Servidores y aplicaciones",
                     "Tester funcional",
                     "Tester automatizada",
                     "SCRUM Master",
                     "Product Owner",
                     "Customer Success",
                     "Gerente de Proyecto",
                     "Analisis de Datos",
                     "Analista de Datos"]

In [5]:
dict_perfiles = {"Backend"                        : "Desarrollador",
                 "Base de datos"                  : "Desarrollador",
                 "Bases de datos"                 : "Desarrollador",
                 "APIs"                           : "Desarrollador",
                 "Mobile"                         : "Desarrollador",
                 "Frontend Movil"                 : "Desarrollador",
                 "Frontend Web"                   : "Desarrollador",
                 "Frontend"                       : "Desarrollador",
                 "Full Stack"                     : "Desarrollador",
                 "Integración"                    : "Desarrollador",
                 "Liferay"                        : "Desarrollador",
                 "Power BI"                       : "Desarrollador",
                 "Software Release Engineer"      : "Devops",
                 "Diseñador Gráfico"              : "Diseñador",
                 "Infraestructura cloud"          : "Infraestructura",
                 "Sistemas Operativos"            : "Infraestructura",
                 "Seguridad"                      : "Infraestructura",
                 "Mantenimiento y Soporte"        : "Infraestructura",
                 "Administrador de Base de datos" : "Infraestructura",
                 "Servidores y aplicaciones"      : "Infraestructura",
                 "Tester funcional"               : "Quality Assurance",
                 "Tester automatizada"            : "Quality Assurance",
                 "SCRUM Master"                   : "Gestión Operativa",
                 "Product Owner"                  : "Gestión Operativa",
                 "Customer Success"               : "Gestión Operativa",
                 "Gerente de Proyecto"            : "Gestión Operativa",
                 "Analisis de Datos"              : "Especialista",
                 "Analista de Datos"              : "Especialista"}

In [2]:
hab1 = AirtableHAB(table_name = "LATAM 001")
hab2 = AirtableHAB(table_name = "LATAM 002")
hab3 = AirtableHAB(table_name = "LATAM 003")

In [12]:
%%time

df1 = hab1.airtable_get()

CPU times: total: 25.7 s
Wall time: 5min 47s


In [10]:
%%time
df2 = hab2.airtable_get()

CPU times: total: 12 s
Wall time: 4min 4s


In [11]:
%%time
df3 = hab3.airtable_get()

CPU times: total: 19.6 s
Wall time: 5min 37s


In [21]:
%%time

df1["experience"] = df1["description"].apply(lambda x : find_years_of_experience(x) if not pd.isna(x) else x)
df1["experience_levels"] = df1["experience"].apply(lambda x : experience_level(x))

df2["experience"] = df2["description"].apply(lambda x : find_years_of_experience(x) if not pd.isna(x) else x)
df2["experience_levels"] = df2["experience"].apply(lambda x : experience_level(x) if not pd.isna(x) else x)

df3["experience"] = df3["description"].apply(lambda x : find_years_of_experience(x) if not pd.isna(x) else x)
df3["experience_levels"] = df3["experience"].apply(lambda x : experience_level(x) if not pd.isna(x) else x)

CPU times: total: 2.77 s
Wall time: 3.5 s


In [22]:
%%time

df1["Especialidad"] = df1["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)
df2["Especialidad"] = df2["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)
df3["Especialidad"] = df3["description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else None)

CPU times: total: 37.7 s
Wall time: 46 s


In [23]:
%%time

df1["Perfil"] = df1["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)
df2["Perfil"] = df2["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)
df3["Perfil"] = df3["description"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles) if not pd.isna(x) else None)

CPU times: total: 4.94 s
Wall time: 6.39 s


In [117]:
df = pd.concat([df1, df2, df3], ignore_index = True)

df.shape

(136360, 53)

In [118]:
df = df.drop_duplicates(subset = "description")

df.shape

(64339, 53)

In [119]:
df = df[(df["salary_max"] > 0) & (df["salary_min"] > 0)]

In [120]:
df.columns

Index(['airtable_id', 'airtable_createdTime', 'job_id', 'title',
       'company_name', 'location', 'source', 'description', 'group',
       'date_posted', 'salary_min', 'salary_max', 'contract_type',
       'experience', 'remote_work', 'group_metrics', 'currency', 'time_lapse',
       'chat_gpt', 'currency_form', 'Grupo', 'Sin salario / Salario ilógico',
       'salary_min_change_usd_form', 'salary_max_change_usd_form',
       'Salario medio', 'Localización', 'Países', 'experience_levels',
       'salary_avg update', 'Salario medio update', 'tech_skills',
       'specialValue', 'skills2', 'Especialidad_Multiple', 'Perfil_Multiple',
       'salary_min_form', 'salary_max_form', 'Vacante NO IT', 'salary_avg',
       'time_lapse_form', 'bonus_form', 'wage_form', 'Salario mínimo',
       'Salario máximo', 'salary_min_changed_usd_extract',
       'salary_max_changed_usd_extract', 'salary_avg_changed_usd_extract',
       'salary_dollars_year', 'Especialidad', 'Perfil',
       'salary_min_cha

In [121]:
df = df[['job_id', 'title', 'company_name', 'location', 'source', 'description', 'date_posted',
'salary_min', 'salary_max', 'contract_type', 'experience', 'remote_work', 'currency', 'time_lapse',
'chat_gpt', 'experience_levels', 'tech_skills', 'Especialidad_Multiple', 'Perfil_Multiple']]

In [122]:
df.columns = ['job_id', 'title', 'company_name', 'location', 'source', 'description', 'date_posted',
'salary_min', 'salary_max', 'contract_type', 'experience', 'remote_work', 'currency', 'time_lapse',
'chat_gpt', 'experience_levels', 'tech_skills', 'Especialidad', 'Perfil']

df

,job_id,title,company_name,location,source,description,date_posted,salary_min,salary_max,contract_type,experience,remote_work,currency,time_lapse,chat_gpt,experience_levels,tech_skills,Especialidad,Perfil
18,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,ANALISTA SEGURIDAD DE LA INFORMACION,ALLIANCE ENTERPRISE SAS,"Bogotá, Bogota, Colombia",Universia,La empresa ALLIANCE ENTERPRISE requiere ANALIS...,2023-05-28,2500000.0,3000000.0,Full-time,NaN,N/A,Peso colombiano (COP),month,Chat-Gpt,NaN,NaN,[Seguridad],[Infraestructura]
35,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,Profesional TI - Analista QA,Perceptual Consultores,"Providencia, Chile",Trabajando.cl,Joven Profesional TI - Analista QA\nImportante...,2023-06-08,650000.0,650000.0,Full-time,NaN,N/A,Peso chileno (CLP),month,Chat-Gpt,NaN,NaN,NaN,NaN
41,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,Técnicos para empresa de telecomunicaciones - ...,Azurdia RH,"Guatemala City, Guatemala",Computrabajo Guatemala,Empresa de telecomunicación tiene las siguient...,2023-06-09,3500.0,3500.0,Full-time,3.0,N/A,GTQ,month,Chat-Gpt,Semi-Senior,NaN,NaN,NaN
42,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,Tecnico de soporte Técnico de Soporte | [CT-973],SISTEMAS SENTRY,"Medellín, Medellin, Antioquia, Colombia",Kit Empleo,Sistemas Sentry busca para su equipo de trabaj...,2023-06-10,1650000.0,1650000.0,Full-time,NaN,N/A,COP,month,Chat-Gpt,NaN,NaN,NaN,NaN
51,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,Técnico en instalaciones,Presea Telecomunicaciones,"Mexico City, CDMX, Mexico",Indeed,EMPRESA LIDER EN TELECOMUNICACIONES\n\n¡¡¡¡SOL...,2023-05-24,8000.0,12000.0,Full-time,NaN,N/A,MXN,month,Chat-Gpt,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136304,eyJqb2JfdGl0bGUiOiJQcm9qZWN0IE1hbmFnZXIsIEdyb3...,"Project Manager, Grounds Maintenance - Fort Ca...",PCSI,"Killeen, TX",ZipRecruiter,PCSI is looking for a Project Manager to lead ...,2023-07-11,62000.0,66000.0,Full-time,2.0,N/A,USD,month,Chat-Gpt,Semi-Senior,NaN,NaN,NaN
136327,eyJqb2JfdGl0bGUiOiJFc3BlY2lhbGlzdGEgZGUgUHJldm...,Especialista de Prevención/seguridad,Distribuidora Liverpool S.A. DE C V.,"Mérida, Yucatan, Mexico",BeBee,Se solicita especialista de prevención de pérd...,2023-07-21,6800.0,6800.0,Full-time,NaN,N/A,MXN,month,Chat-Gpt,NaN,NaN,[Seguridad],NaN
136331,eyJqb2JfdGl0bGUiOiJFU1BFQ0lBTElTVEEgRU4gQUxNQU...,ESPECIALISTA EN ALMACÉN,DAL SOLUCIONES,"Naucalpan de Juárez, State of Mexico, Mexico",Indeed,En DAL Soluciones buscamos ESPECIALISTA EN ALM...,2023-07-27,9000.0,9000.0,Full-time,NaN,N/A,MXN,month,Chat-Gpt,NaN,NaN,NaN,NaN
136336,eyJqb2JfdGl0bGUiOiJEaXNlw7FhZG9yIHdlYiB5IG11bH...,Diseñador web y multimedia,Impulso Marketing,"Guadalajara, Jalisco, Mexico",Indeed,Agencia de marketing solicita diseñador con co...,2023-07-04,10000.0,10000.0,Full-time,NaN,N/A,MXN,month,Chat-Gpt,NaN,NaN,NaN,NaN


In [280]:
df["title_description"] = df["title"] + " " + df["description"]

df["Especialidad_Multiple"] = df["title_description"].apply(lambda x : get_especialidad(x, list_especialidad = list_especialidad) if not pd.isna(x) else x)

In [281]:
for x, y in zip(df["Especialidad"], df["Especialidad_Multiple"]):
    print(f"{x} // {y}")

['Seguridad'] // ['Seguridad']
None // []
None // []
None // []
None // []
None // []
None // []
['Seguridad', 'Sistemas Operativos'] // ['Sistemas Operativos', 'Seguridad']
None // []
['Mobile'] // ['Mobile']
['Diseñador Gráfico'] // ['Diseñador Gráfico']
['Integración'] // ['Integración']
['Integración'] // ['Integración']
None // []
None // []
['Sistemas Operativos'] // ['Sistemas Operativos']
['Diseñador Gráfico'] // ['Diseñador Gráfico']
['Seguridad'] // ['Bases de datos', 'Seguridad']
None // []
None // []
['Seguridad'] // ['Bases de datos', 'Seguridad']
None // []
['Sistemas Operativos'] // ['Sistemas Operativos']
None // []
['Base de datos'] // ['Base de datos', 'Bases de datos']
['Integración'] // ['Integración']
['Full Stack', 'Backend', 'APIs'] // ['APIs', 'Backend']
None // []
None // []
None // []
['Backend'] // ['Backend']
['Power BI'] // ['Analista de Datos', 'Power BI']
None // []
None // ['Analista de Datos']
None // ['Bases de datos']
None // []
None // []
['Seguridad

['Seguridad'] // ['Seguridad']
None // []
None // []
None // []
None // ['Bases de datos']
None // ['Bases de datos', 'Analista de Datos']
['Seguridad'] // ['Seguridad']
None // []
None // []
None // ['Bases de datos']
['Base de datos'] // ['Base de datos', 'Analista de Datos']
['Backend'] // ['Backend']
['Seguridad'] // ['Seguridad']
None // []
['Backend'] // ['Backend']
None // []
None // []
None // []
None // []
None // []
['Integración'] // ['Bases de datos', 'Analista de Datos', 'Integración']
None // []
['Integración'] // ['Bases de datos', 'Integración']
None // []
None // []
None // []
None // []
None // []
None // ['Bases de datos']
None // []
None // []
None // []
None // ['Mobile']
None // []
None // []
['Full Stack', 'Integración', 'APIs'] // ['Bases de datos', 'APIs', 'Integración']
['Integración'] // ['Bases de datos', 'Integración']
None // []
['Base de datos'] // ['Base de datos']
None // []
None // []
None // []
None // ['Bases de datos']
None // []
None // []
None // 

['Seguridad', 'Infraestructura cloud'] // ['Seguridad', 'Infraestructura cloud']
None // []
None // []
None // ['Bases de datos']
['Seguridad'] // ['Seguridad']
['Seguridad'] // ['Seguridad']
['Base de datos'] // ['Base de datos', 'Analista de Datos']
None // []
None // []
None // []
None // []
None // []
None // []
None // []
None // ['Analista de Datos']
None // []
None // []
None // []
None // []
['Base de datos'] // ['Base de datos']
None // []
['Integración'] // ['Bases de datos', 'Integración', 'Backend']
['Seguridad'] // ['Seguridad']
None // []
['Seguridad'] // ['Seguridad']
None // []
None // []
['Sistemas Operativos'] // ['Sistemas Operativos']
['Backend', 'Base de datos', 'APIs'] // ['Base de datos', 'APIs', 'Backend']
['Integración'] // ['Bases de datos', 'Integración']
['Servidores y aplicaciones', 'Infraestructura cloud', 'Integración'] // ['Infraestructura cloud', 'Servidores y aplicaciones', 'Integración']
None // []
None // ['Analista de Datos']
['Seguridad'] // ['Segu

In [282]:
df["Perfil"] = df["Especialidad_Multiple"].apply(lambda x : get_perfil(x, dict_perfiles = dict_perfiles))

In [283]:
df["Perfil"]

0        [Infraestructura]
1                       []
2                       []
3                       []
4                       []
               ...        
10658                   []
10659    [Infraestructura]
10660                   []
10661                   []
10662      [Desarrollador]
Name: Perfil, Length: 10663, dtype: object

In [234]:
df.isna().sum()

job_id                      0
title                       0
company_name                0
location                    1
source                      0
description                 0
date_posted                 0
salary_min                  0
salary_max                  0
contract_type               0
experience               8242
remote_work                 0
currency                  117
time_lapse                571
chat_gpt                    0
experience_levels        8242
tech_skills              6189
Especialidad             6106
Perfil                      0
title_description           0
Especialidad_Multipe        0
Especialidad_Multiple       0
dtype: int64

In [124]:
def get_remote_work(string):
    
    resultados = list()
    
    if "remoto" in string or "remote work" in string:
        
        resultados.append("Remoto")
        
    elif "hibrido" in string or "hybrid" in string:
        
        resultados.append("Hibrido")
        
    elif "presencial" in string :
        
        resultados.append("Presencial")
        
    return resultados

In [125]:
df["remote_work"] = df["description"].apply(lambda x : get_remote_work(x))

In [126]:
def get_iso_currency(string):
    
    if len(string) == 3:
        return string

    regex = r'\((\w+)\)'

    resultados = re.findall(regex, string)
    
    if len(resultados) > 0:
        return resultados[0]
    else:
        return None


In [127]:
df["currency"] = df["currency"].apply(lambda x : get_iso_currency(x) if not pd.isna(x) else x)

In [128]:
df.replace("N/A", np.nan, inplace = True)

In [129]:
df.reset_index(drop = True, inplace = True) 

In [130]:
df.head()

,job_id,title,company_name,location,source,description,date_posted,salary_min,salary_max,contract_type,experience,remote_work,currency,time_lapse,chat_gpt,experience_levels,tech_skills,Especialidad,Perfil
0,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,ANALISTA SEGURIDAD DE LA INFORMACION,ALLIANCE ENTERPRISE SAS,"Bogotá, Bogota, Colombia",Universia,La empresa ALLIANCE ENTERPRISE requiere ANALIS...,2023-05-28,2500000.0,3000000.0,Full-time,NaN,[],COP,month,Chat-Gpt,NaN,NaN,[Seguridad],[Infraestructura]
1,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,Profesional TI - Analista QA,Perceptual Consultores,"Providencia, Chile",Trabajando.cl,Joven Profesional TI - Analista QA\nImportante...,2023-06-08,650000.0,650000.0,Full-time,NaN,[],CLP,month,Chat-Gpt,NaN,NaN,NaN,NaN
2,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,Técnicos para empresa de telecomunicaciones - ...,Azurdia RH,"Guatemala City, Guatemala",Computrabajo Guatemala,Empresa de telecomunicación tiene las siguient...,2023-06-09,3500.0,3500.0,Full-time,3.0,[],GTQ,month,Chat-Gpt,Semi-Senior,NaN,NaN,NaN
3,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,Tecnico de soporte Técnico de Soporte | [CT-973],SISTEMAS SENTRY,"Medellín, Medellin, Antioquia, Colombia",Kit Empleo,Sistemas Sentry busca para su equipo de trabaj...,2023-06-10,1650000.0,1650000.0,Full-time,NaN,[],COP,month,Chat-Gpt,NaN,NaN,NaN,NaN
4,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,Técnico en instalaciones,Presea Telecomunicaciones,"Mexico City, CDMX, Mexico",Indeed,EMPRESA LIDER EN TELECOMUNICACIONES\n\n¡¡¡¡SOL...,2023-05-24,8000.0,12000.0,Full-time,NaN,[Presencial],MXN,month,Chat-Gpt,NaN,NaN,NaN,NaN


In [151]:
def clean_lista(lista):
    
    try:
        return lista if len(lista) > 0 else None
    except:
        return None
    
def clean2(x):
    
    if str(x) == "nan":
        return None
    
    else:
        return x

In [241]:
df["remote_work"] = df["remote_work"].apply(lambda x : clean_lista(x))
df["Especialidad"] = df["Especialidad"].apply(lambda x : clean_lista(x))
df["Perfil"] = df["Perfil"].apply(lambda x : clean_lista(x))


In [157]:
for col in df.columns:

    df[col] = df[col].apply(lambda x : clean2(x))


In [158]:
df

,job_id,title,company_name,location,source,description,date_posted,salary_min,salary_max,contract_type,experience,remote_work,currency,time_lapse,chat_gpt,experience_levels,tech_skills,Especialidad,Perfil
0,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,ANALISTA SEGURIDAD DE LA INFORMACION,ALLIANCE ENTERPRISE SAS,"Bogotá, Bogota, Colombia",Universia,La empresa ALLIANCE ENTERPRISE requiere ANALIS...,2023-05-28,2500000.0,3000000.0,Full-time,NaN,None,COP,month,Chat-Gpt,None,None,[Seguridad],[Infraestructura]
1,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,Profesional TI - Analista QA,Perceptual Consultores,"Providencia, Chile",Trabajando.cl,Joven Profesional TI - Analista QA\nImportante...,2023-06-08,650000.0,650000.0,Full-time,NaN,None,CLP,month,Chat-Gpt,None,None,None,None
2,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,Técnicos para empresa de telecomunicaciones - ...,Azurdia RH,"Guatemala City, Guatemala",Computrabajo Guatemala,Empresa de telecomunicación tiene las siguient...,2023-06-09,3500.0,3500.0,Full-time,3.0,None,GTQ,month,Chat-Gpt,Semi-Senior,None,None,None
3,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,Tecnico de soporte Técnico de Soporte | [CT-973],SISTEMAS SENTRY,"Medellín, Medellin, Antioquia, Colombia",Kit Empleo,Sistemas Sentry busca para su equipo de trabaj...,2023-06-10,1650000.0,1650000.0,Full-time,NaN,None,COP,month,Chat-Gpt,None,None,None,None
4,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,Técnico en instalaciones,Presea Telecomunicaciones,"Mexico City, CDMX, Mexico",Indeed,EMPRESA LIDER EN TELECOMUNICACIONES\n\n¡¡¡¡SOL...,2023-05-24,8000.0,12000.0,Full-time,NaN,[Presencial],MXN,month,Chat-Gpt,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10658,eyJqb2JfdGl0bGUiOiJQcm9qZWN0IE1hbmFnZXIsIEdyb3...,"Project Manager, Grounds Maintenance - Fort Ca...",PCSI,"Killeen, TX",ZipRecruiter,PCSI is looking for a Project Manager to lead ...,2023-07-11,62000.0,66000.0,Full-time,2.0,None,USD,month,Chat-Gpt,Semi-Senior,None,None,None
10659,eyJqb2JfdGl0bGUiOiJFc3BlY2lhbGlzdGEgZGUgUHJldm...,Especialista de Prevención/seguridad,Distribuidora Liverpool S.A. DE C V.,"Mérida, Yucatan, Mexico",BeBee,Se solicita especialista de prevención de pérd...,2023-07-21,6800.0,6800.0,Full-time,NaN,None,MXN,month,Chat-Gpt,None,None,[Seguridad],None
10660,eyJqb2JfdGl0bGUiOiJFU1BFQ0lBTElTVEEgRU4gQUxNQU...,ESPECIALISTA EN ALMACÉN,DAL SOLUCIONES,"Naucalpan de Juárez, State of Mexico, Mexico",Indeed,En DAL Soluciones buscamos ESPECIALISTA EN ALM...,2023-07-27,9000.0,9000.0,Full-time,NaN,None,MXN,month,Chat-Gpt,None,None,None,None
10661,eyJqb2JfdGl0bGUiOiJEaXNlw7FhZG9yIHdlYiB5IG11bH...,Diseñador web y multimedia,Impulso Marketing,"Guadalajara, Jalisco, Mexico",Indeed,Agencia de marketing solicita diseñador con co...,2023-07-04,10000.0,10000.0,Full-time,NaN,None,MXN,month,Chat-Gpt,None,None,None,None


In [174]:
df.isna().sum()

job_id                  0
title                   0
company_name            0
location                1
source                  0
description             0
date_posted             0
salary_min              0
salary_max              0
contract_type         228
experience           8242
remote_work          8217
currency              117
time_lapse            571
chat_gpt                0
experience_levels    8242
tech_skills          6189
Especialidad         6106
Perfil               8610
dtype: int64

In [136]:
hab0 = AirtableHAB(table_name = "LATAM 000")

hab0.endpoint

'https://api.airtable.com/v0/appU0CLDju62P5dKe/tbl6S7ztip0gcv241'

In [161]:
datos_subir = [{"fields" : {k : v if str(v) != "nan" else None for k, v in df.iloc[i, :].to_dict().items()}} for i in range(df.shape[0])]

datos_subir

[{'fields': {'job_id': 'eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgREUgTEEgSU5GT1JNQUNJT04iLCJodGlkb2NpZCI6Ik9md3NsaVdJZTh3QUFBQUFBQUFBQUE9PSIsInV1bGUiOiJ3K0NBSVFJQ0lJUTI5c2IyMWlhV0UiLCJmYyI6IkVvd0NDc3dCUVVWek4ycE9VblpFT1VwT1gwZFNTVjlXUjNJeFZsODBiVTlzTmxSTExWVjFNRWx5ZDNkS1JqTk5iMFJLY0ZKV0xYTmxXVVk0Um1Zd2MwNURNbnBVWmpaUlIzZFRSamhzWmkweVpWcEdSbGRHWnpoQ1pYQjNaMDlQUVcxR0xYQklVMmQ0VWsxeU9VSTNNRkUyTlZOVlpVdExRbFJvYzFKNmIzVllWSFZGWkhWamIwVmhWM05tUzI5U2RXRkxZWFZSV2tkR2JYTm1RVXRMUWtWVGFEUlZaek5HZFVaa1FVdFRkMjR5V0VWc1dXVTFTM2R1UkVGeFZ6SjRZbUo1ZUdGMVN6UkNPV3B6TUhOa0xVaGhFaGRDVlVkVVdrNTFiazVsUzI1d2RGRlFjRXBUZVhkQlJSb2lRVTh0TUhKc05HaFRNbWMxY2xKbVJHbHJSbkZ2YUY4MlkxcEhka2REVkdoTVVRIiwiZmN2IjoiMyIsImZjX2lkIjoiZmNfNSIsImFwcGx5X2xpbmsiOnsidGl0bGUiOiJBcHBseSBvbiBVbml2ZXJzaWEiLCJsaW5rIjoiaHR0cHM6Ly93d3cudW5pdmVyc2lhLm5ldC9jby9lbXBsZW8vNzA3ODE3ZjItZDkzOS00ZjIzLTgwMzAtYWJjYjllYWE1YjQwL2FuYWxpc3RhLXNlZ3VyaWRhZC1kZS1sYS1pbmZvcm1hY2lvbi0xNjg1MTE3OTk3NjcyNTM1Lmh0bWw/b3JpZ2luPVVuaXZlcnNpZGFkK2RlK1NhbitCdWVuYXZl

In [171]:
for i in range(0, df.shape[0], 10):
    
    data = {"records" : datos_subir[i : i + 10],
            "typecast" : True}
    
    response = requests.post(url = hab0.endpoint, json = data, headers = hab0.headers) # POST
    
    print(response.status_code)

422
422
422
422


KeyboardInterrupt: 

,fldOpA2WttGN9hR0k,fldGvHnwLDBTJo4tG,fldjwd8p6v46W3r9Q,fldanIL0AD99VUPtb,fldJc8q1va2LAyi9A,fldz5pG9paG0ZF5GE,fldkwtppfHSSq8AI8,fldRDgLQqsG2LlX84,fldPuGFiGQqftMjHf,fldJUw9pLgNxQStvA,flduojBYTuU4zIf5B,fldt2Q0rgrQr1Tn1u,fldcVZu2L7d07W7Ss,fldQgTQbqG7Gmbybq,fldmYVz9dynfMuBhF,fldSoWvhicZwwNd7M,fld7Bg7mFo5Rqei8P,fldXjAOksOsuQr0gO,fldB4PegiXBezLM4z
0,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,ANALISTA SEGURIDAD DE LA INFORMACION,ALLIANCE ENTERPRISE SAS,"Bogotá, Bogota, Colombia",Universia,La empresa ALLIANCE ENTERPRISE requiere ANALIS...,2023-05-28,2500000.0,3000000.0,Full-time,,N/A,COP,month,Chat-Gpt,,,[Seguridad],[Infraestructura]
1,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,Profesional TI - Analista QA,Perceptual Consultores,"Providencia, Chile",Trabajando.cl,Joven Profesional TI - Analista QA\nImportante...,2023-06-08,650000.0,650000.0,Full-time,,N/A,CLP,month,Chat-Gpt,,,,
2,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,Técnicos para empresa de telecomunicaciones - ...,Azurdia RH,"Guatemala City, Guatemala",Computrabajo Guatemala,Empresa de telecomunicación tiene las siguient...,2023-06-09,3500.0,3500.0,Full-time,3.0,N/A,GTQ,month,Chat-Gpt,Semi-Senior,,,
3,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,Tecnico de soporte Técnico de Soporte | [CT-973],SISTEMAS SENTRY,"Medellín, Medellin, Antioquia, Colombia",Kit Empleo,Sistemas Sentry busca para su equipo de trabaj...,2023-06-10,1650000.0,1650000.0,Full-time,,N/A,COP,month,Chat-Gpt,,,,
4,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,Técnico en instalaciones,Presea Telecomunicaciones,"Mexico City, CDMX, Mexico",Indeed,EMPRESA LIDER EN TELECOMUNICACIONES\n\n¡¡¡¡SOL...,2023-05-24,8000.0,12000.0,Full-time,,['Presencial'],MXN,month,Chat-Gpt,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10658,eyJqb2JfdGl0bGUiOiJQcm9qZWN0IE1hbmFnZXIsIEdyb3...,"Project Manager, Grounds Maintenance - Fort Ca...",PCSI,"Killeen, TX",ZipRecruiter,PCSI is looking for a Project Manager to lead ...,2023-07-11,62000.0,66000.0,Full-time,2.0,N/A,USD,month,Chat-Gpt,Semi-Senior,,,
10659,eyJqb2JfdGl0bGUiOiJFc3BlY2lhbGlzdGEgZGUgUHJldm...,Especialista de Prevención/seguridad,Distribuidora Liverpool S.A. DE C V.,"Mérida, Yucatan, Mexico",BeBee,Se solicita especialista de prevención de pérd...,2023-07-21,6800.0,6800.0,Full-time,,N/A,MXN,month,Chat-Gpt,,,[Seguridad],
10660,eyJqb2JfdGl0bGUiOiJFU1BFQ0lBTElTVEEgRU4gQUxNQU...,ESPECIALISTA EN ALMACÉN,DAL SOLUCIONES,"Naucalpan de Juárez, State of Mexico, Mexico",Indeed,En DAL Soluciones buscamos ESPECIALISTA EN ALM...,2023-07-27,9000.0,9000.0,Full-time,,N/A,MXN,month,Chat-Gpt,,,,
10661,eyJqb2JfdGl0bGUiOiJEaXNlw7FhZG9yIHdlYiB5IG11bH...,Diseñador web y multimedia,Impulso Marketing,"Guadalajara, Jalisco, Mexico",Indeed,Agencia de marketing solicita diseñador con co...,2023-07-04,10000.0,10000.0,Full-time,,N/A,MXN,month,Chat-Gpt,,,,


In [179]:
hab0.airtable_post(hab0.fill_na(df))

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [183]:
df["title"]

0                     ANALISTA SEGURIDAD DE LA INFORMACION
1                             Profesional TI - Analista QA
2        Técnicos para empresa de telecomunicaciones - ...
3         Tecnico de soporte Técnico de Soporte | [CT-973]
4                                 Técnico en instalaciones
                               ...                        
10658    Project Manager, Grounds Maintenance - Fort Ca...
10659                 Especialista de Prevención/seguridad
10660                              ESPECIALISTA EN ALMACÉN
10661                           Diseñador web y multimedia
10662           Desarrollador Java Microservicios - Remoto
Name: title, Length: 10663, dtype: object

In [184]:
df__ = pd.read_csv("latam 004.csv")

df___ = pd.concat([df1, df2, df3, df__], ignore_index = True)


print(df__.shape)

print(df___.drop_duplicates("description").shape)

C:\Users\danie\AppData\Local\Temp\ipykernel_7420\2647756935.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df__ = pd.read_csv("latam 004.csv")


(57729, 18)
(121133, 64)


In [242]:
%%time
df_air = hab0.airtable_get()

CPU times: total: 953 ms
Wall time: 1min 8s


In [245]:
df.columns

Index(['job_id', 'title', 'company_name', 'location', 'source', 'description',
       'date_posted', 'salary_min', 'salary_max', 'contract_type',
       'experience', 'remote_work', 'currency', 'time_lapse', 'chat_gpt',
       'experience_levels', 'tech_skills', 'Especialidad', 'Perfil',
       'title_description', 'Especialidad_Multipe', 'Especialidad_Multiple'],
      dtype='object')

In [284]:
df_merge = df[["job_id", "Perfil", "Especialidad_Multiple"]]

df_merge.columns = ["job_id", "Perfil", "Especialidad"]

df_merge

,job_id,Perfil,Especialidad
0,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,[Infraestructura],[Seguridad]
1,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,[],[]
2,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,[],[]
3,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,[],[]
4,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,[],[]
...,...,...,...
10658,eyJqb2JfdGl0bGUiOiJQcm9qZWN0IE1hbmFnZXIsIEdyb3...,[],[]
10659,eyJqb2JfdGl0bGUiOiJFc3BlY2lhbGlzdGEgZGUgUHJldm...,[Infraestructura],[Seguridad]
10660,eyJqb2JfdGl0bGUiOiJFU1BFQ0lBTElTVEEgRU4gQUxNQU...,[],[]
10661,eyJqb2JfdGl0bGUiOiJEaXNlw7FhZG9yIHdlYiB5IG11bH...,[],[]


In [247]:
df_air.columns

Index(['airtable_id', 'airtable_createdTime', 'job_id', 'salary_min',
       'salary_max', 'remote_work', 'title', 'company_name', 'location',
       'source', 'description', 'date_posted', 'contract_type', 'currency',
       'time_lapse', 'chat_gpt', 'salary_min_usd', 'experience_levels',
       'Especialidad', 'tech_skills', 'experience', 'Perfil'],
      dtype='object')

In [285]:
df_air = df_air.drop(["Perfil", "Especialidad"], axis = 1).merge(df_merge, on = "job_id")

In [286]:
df___1 = df_air[["airtable_id", "Perfil", "Especialidad"]]

In [287]:
df___1.columns = ["id", "Perfil", "Especialidad"]

In [288]:
df___1

,id,Perfil,Especialidad
0,rec009VSLt1q03mw8,[],[]
1,rec00sIHIJw6JQOzm,"[Desarrollador, Infraestructura]","[Base de datos, Seguridad, Administrador de Ba..."
2,rec00sO2LIHaKX31x,[Infraestructura],"[Sistemas Operativos, Seguridad]"
3,rec00wMiDE2kQzNtY,[],[]
4,rec018amGncdv5biL,[Desarrollador],[Base de datos]
...,...,...,...
10642,reczvBGVljpZfyc8y,[],[]
10643,reczweTjz1j9Pkq84,[Desarrollador],"[Base de datos, Backend]"
10644,reczx3yYYHRG0N4lV,[],[]
10645,reczxvy0VabfaEFuI,[],[]


In [271]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
    break

In [275]:
vacio = list()

for row in data.values:
    
    id_ = row[0]
    
    Perfil, Especialidad = row[1], row[2]
    
    vacio.append({"id" : id_, "fields" : {"Perfil" : Perfil, "Especialidad" : Especialidad}})
    
    
vacio

[{'id': 'rec009VSLt1q03mw8', 'fields': {'Perfil': None, 'Especialidad': []}},
 {'id': 'rec00sIHIJw6JQOzm',
  'fields': {'Perfil': ['Desarrollador', 'Infraestructura'],
   'Especialidad': ['Base de datos',
    'Seguridad',
    'Administrador de Base de datos']}},
 {'id': 'rec00sO2LIHaKX31x',
  'fields': {'Perfil': ['Infraestructura'],
   'Especialidad': ['Sistemas Operativos', 'Seguridad']}},
 {'id': 'rec00wMiDE2kQzNtY', 'fields': {'Perfil': None, 'Especialidad': []}},
 {'id': 'rec018amGncdv5biL',
  'fields': {'Perfil': ['Desarrollador'], 'Especialidad': ['Base de datos']}},
 {'id': 'rec019LNEbcvXGi7H', 'fields': {'Perfil': None, 'Especialidad': []}},
 {'id': 'rec01K7bMtQjwg9Kx', 'fields': {'Perfil': None, 'Especialidad': []}},
 {'id': 'rec01SRq6IOS7lLMd',
  'fields': {'Perfil': ['Desarrollador', 'Infraestructura'],
   'Especialidad': ['Base de datos', 'Seguridad']}},
 {'id': 'rec01hsIcvc2xswu6', 'fields': {'Perfil': None, 'Especialidad': []}},
 {'id': 'rec02GTpsrpi5dykM', 'fields': {'P

In [289]:
for i in range(0, df___1.shape[0], 10):
    
    data = df___1.iloc[i : i + 10].copy()
    
#     data = [{"id" : data.iloc[x, 0], "fields" : {k : v if not pd.isna(v) else None for k, v in data.iloc[x, 1:].to_dict().items()}} for x in range(len(data))]
    
    
    vacio = list()

    for row in data.values:

        id_ = row[0]

        Perfil, Especialidad = row[1], row[2]

        vacio.append({"id" : id_, "fields" : {"Perfil" : Perfil, "Especialidad" : Especialidad}})
    
    data = {"records" : vacio, "typecast" : True}

    response = requests.patch(url = hab0.endpoint, json = data, headers = hab0.headers)
    
    print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [291]:
df004 = pd.read_csv("raw_extract_n_1.csv")
df004

,Unnamed: 0,job_id,reduce_description,salary_min_gpt_0.8.1,salary_max_gpt_0.8.1,currency_gpt_0.8.1,time_lapse_gpt_0.8.1,source_0.8.1,token_count_sent_chat_gpt_0.8.1,token_count_original_description,...,extensions,posted_at,schedule_type,work_from_home,country_search,salary,commute_time,date_posted,tech_skills,description_normalized
0,0,eyJqb2JfdGl0bGUiOiJTZW5pb3IgU29mdHdhcmUgRW5naW...,got a 77% flex score. technology discipline 5+...,100000.0,125000.0,USD,year,Chat-GptChat-Gpt-[CACHED],686.0,1036,...,"['22 hours ago', 'Work from home', 'Full-time']",0 days 22:00:00,Full-time,True,Argentina,NaN,NaN,2023-08-02,"['AWS', 'GIT', 'MongoDB', 'Strategy']",This a Full Remote job the offer is available ...
1,1,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBEZXZlbG9wZXIgKF...,100% remote job in engineering. 3+ years of wo...,0.0,0.0,,,Chat-GptChat-Gpt-[CACHED],622.0,557,...,"['21 days ago', 'Work from home', 'Full-time']",21 days 00:00:00,Full-time,True,Argentina,NaN,NaN,2023-06-21,[],100% remote job Software Developer | ioet | Ar...
2,2,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBEZXZlbG9wZXIgKF...,technologies: python 3.7 and higher alembicc. ...,0.0,0.0,,,Chat-GptChat-Gpt-[CACHED],727.0,368,...,"['Work from home', 'Full-time', 'No degree men...",NaN,Full-time,True,Argentina,NaN,NaN,2023-08-02,"['Flask', 'Python', 'SQL']",👉 Summary We are looking for a new GIGAnte to ...
3,3,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRnVsbC1zdGFjayBzb2...,business over 30+ years. we team consideration...,0.0,0.0,,,Chat-GptChat-Gpt-[CACHED],645.0,411,...,"['11 hours ago', 'Work from home', 'Full-time'...",0 days 11:00:00,Full-time,True,Argentina,NaN,NaN,2023-08-02,"['AWS', 'Figma', 'Python', 'REST', 'SQL']",About The Position Senior Full Stack Software ...
4,4,eyJqb2JfdGl0bGUiOiJSZWFjdCBTb2Z0d2FyZSBEZXZlbG...,at least 3 years of oowlish paradise 4 days/ye...,0.0,0.0,,,Chat-GptChat-Gpt-[CACHED],625.0,538,...,"['23 days ago', 'Full-time', 'No degree mentio...",23 days 00:00:00,Full-time,NaN,Argentina,NaN,NaN,2023-06-17,['HTML'],Be part of our team 🚀💼 Are you ready to launch...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,eyJqb2JfdGl0bGUiOiJNdXJleCBGcm9udCBPZmZpY2UgQn...,"are: since 1998, exadel has currently has 2000...",0.0,0.0,,,Chat-Gpt,648.0,238,...,['Full-time'],NaN,Full-time,NaN,Brazil,NaN,NaN,2023-08-02,['Jenkins'],Work at Exadel – Who We Are Since 1998 Exadel ...
4997,4997,eyJqb2JfdGl0bGUiOiJJVCBCdXNpbmVzcyBBbmFseXN0Ii...,"the next 12 months, this help reduce co2 emiss...",12.0,12.0,-,month,Chat-Gpt,633.0,600,...,"['21 days ago', 'Full-time']",21 days 00:00:00,Full-time,NaN,Brazil,NaN,NaN,2023-06-21,"['Azure', 'Jira']",Req ID Sorocaba Brazil Job description ZF is a...
4998,4998,eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBBbmFseXN0IiwiaH...,"(at least 5 years) as dss, iso 20022). strong ...",0.0,0.0,,,Chat-Gpt,640.0,718,...,['Full-time'],NaN,Full-time,NaN,Brazil,NaN,NaN,2023-08-02,[],It's fun to work in a company where people tru...
4999,4999,eyJqb2JfdGl0bGUiOiJCdXNpbmVzcyBBbmFseXN0IEludG...,"on june 1st, 2023. descricao partir de 1o de j...",0.0,0.0,,,Chat-Gpt,755.0,715,...,"['10 days ago', 'Internship']",10 days 00:00:00,Internship,NaN,Brazil,NaN,NaN,2023-07-13,"['Excel', 'SQL']",At Amazon we are working to be the most custom...


In [292]:
df004.columns

Index(['Unnamed: 0', 'job_id', 'reduce_description', 'salary_min_gpt_0.8.1',
       'salary_max_gpt_0.8.1', 'currency_gpt_0.8.1', 'time_lapse_gpt_0.8.1',
       'source_0.8.1', 'token_count_sent_chat_gpt_0.8.1',
       'token_count_original_description',
       'token_rate_sent_original_description', 'gpt_cost_0.8.1', 'title',
       'company_name', 'location', 'via', 'description', 'job_highlights',
       'related_links', 'thumbnail', 'extensions', 'posted_at',
       'schedule_type', 'work_from_home', 'country_search', 'salary',
       'commute_time', 'date_posted', 'tech_skills', 'description_normalized'],
      dtype='object')

In [294]:
df

,job_id,title,company_name,location,source,description,date_posted,salary_min,salary_max,contract_type,...,currency,time_lapse,chat_gpt,experience_levels,tech_skills,Especialidad,Perfil,title_description,Especialidad_Multipe,Especialidad_Multiple
0,eyJqb2JfdGl0bGUiOiJBTkFMSVNUQSBTRUdVUklEQUQgRE...,ANALISTA SEGURIDAD DE LA INFORMACION,ALLIANCE ENTERPRISE SAS,"Bogotá, Bogota, Colombia",Universia,La empresa ALLIANCE ENTERPRISE requiere ANALIS...,2023-05-28,2500000.0,3000000.0,Full-time,...,COP,month,Chat-Gpt,None,None,[Seguridad],[Infraestructura],ANALISTA SEGURIDAD DE LA INFORMACION La empres...,[Seguridad],[Seguridad]
1,eyJqb2JfdGl0bGUiOiJQcm9mZXNpb25hbCBUSSAtIEFuYW...,Profesional TI - Analista QA,Perceptual Consultores,"Providencia, Chile",Trabajando.cl,Joven Profesional TI - Analista QA\nImportante...,2023-06-08,650000.0,650000.0,Full-time,...,CLP,month,Chat-Gpt,None,None,None,[],Profesional TI - Analista QA Joven Profesional...,[],[]
2,eyJqb2JfdGl0bGUiOiJUw6ljbmljb3MgcGFyYSBlbXByZX...,Técnicos para empresa de telecomunicaciones - ...,Azurdia RH,"Guatemala City, Guatemala",Computrabajo Guatemala,Empresa de telecomunicación tiene las siguient...,2023-06-09,3500.0,3500.0,Full-time,...,GTQ,month,Chat-Gpt,Semi-Senior,None,None,[],Técnicos para empresa de telecomunicaciones - ...,[],[]
3,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgVM...,Tecnico de soporte Técnico de Soporte | [CT-973],SISTEMAS SENTRY,"Medellín, Medellin, Antioquia, Colombia",Kit Empleo,Sistemas Sentry busca para su equipo de trabaj...,2023-06-10,1650000.0,1650000.0,Full-time,...,COP,month,Chat-Gpt,None,None,None,[],Tecnico de soporte Técnico de Soporte | [CT-97...,[],[]
4,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBlbiBpbnN0YWxhY2...,Técnico en instalaciones,Presea Telecomunicaciones,"Mexico City, CDMX, Mexico",Indeed,EMPRESA LIDER EN TELECOMUNICACIONES\n\n¡¡¡¡SOL...,2023-05-24,8000.0,12000.0,Full-time,...,MXN,month,Chat-Gpt,None,None,None,[],Técnico en instalaciones EMPRESA LIDER EN TELE...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10658,eyJqb2JfdGl0bGUiOiJQcm9qZWN0IE1hbmFnZXIsIEdyb3...,"Project Manager, Grounds Maintenance - Fort Ca...",PCSI,"Killeen, TX",ZipRecruiter,PCSI is looking for a Project Manager to lead ...,2023-07-11,62000.0,66000.0,Full-time,...,USD,month,Chat-Gpt,Semi-Senior,None,None,[],"Project Manager, Grounds Maintenance - Fort Ca...",[],[]
10659,eyJqb2JfdGl0bGUiOiJFc3BlY2lhbGlzdGEgZGUgUHJldm...,Especialista de Prevención/seguridad,Distribuidora Liverpool S.A. DE C V.,"Mérida, Yucatan, Mexico",BeBee,Se solicita especialista de prevención de pérd...,2023-07-21,6800.0,6800.0,Full-time,...,MXN,month,Chat-Gpt,None,None,[Seguridad],[Infraestructura],Especialista de Prevención/seguridad Se solici...,[Seguridad],[Seguridad]
10660,eyJqb2JfdGl0bGUiOiJFU1BFQ0lBTElTVEEgRU4gQUxNQU...,ESPECIALISTA EN ALMACÉN,DAL SOLUCIONES,"Naucalpan de Juárez, State of Mexico, Mexico",Indeed,En DAL Soluciones buscamos ESPECIALISTA EN ALM...,2023-07-27,9000.0,9000.0,Full-time,...,MXN,month,Chat-Gpt,None,None,None,[],ESPECIALISTA EN ALMACÉN En DAL Soluciones busc...,[],[]
10661,eyJqb2JfdGl0bGUiOiJEaXNlw7FhZG9yIHdlYiB5IG11bH...,Diseñador web y multimedia,Impulso Marketing,"Guadalajara, Jalisco, Mexico",Indeed,Agencia de marketing solicita diseñador con co...,2023-07-04,10000.0,10000.0,Full-time,...,MXN,month,Chat-Gpt,None,None,None,[],Diseñador web y multimedia Agencia de marketin...,[],[]


In [ ]:
################################################################################################################################

In [7]:
%%time

hab_purged = AirtableHAB(table_name = "LATAM 000 purged")

df = hab_purged.airtable_get()

CPU times: total: 4.34 s
Wall time: 1min 46s


In [8]:
with open(file = "locations_airtable.json", mode = "br") as file:
    locations_cargados = pickle.load(file)

UnpicklingError: invalid load key, '{'.